# Texture Analysis Notebook for PNG CT Images

This notebook provides an easy-to-use tool for performing texture analysis on PNG medical images (assumed to be CT images).
It computes texture features using the Gray-Level Co-occurrence Matrix (GLCM).

**Features:**

- **Texture Features:**
  Computes the following GLCM-based features:
  - **Contrast:** Quantifies local intensity variations.
  - **Correlation:** Measures the linear dependency between neighboring pixels.
  - **Energy:** Indicates textural uniformity.
  - **Homogeneity:** Reflects the closeness of the distribution of elements in the GLCM to its diagonal.

- **Basic Statistics:**
  Calculates the mean, standard deviation, and median of the image intensities.

- **Black Frame Detection:**
  Automatically skips images that are completely black (i.e. maximum pixel value is 0).

- **Output:**
  Saves the analysis results in a CSV file and displays the images along with the computed metrics.

**Instructions:**

1. Adjust the `image_directory` variable (the folder containing your PNG images).
2. Run the notebook cells in order.
3. Converted images will be stored in a subfolder named `CT_converted`.
4. The texture analysis results will be saved to `texture_analysis_results.csv`.

---

## Explanation of Texture Analysis Methods

The notebook computes several texture features based on the Gray-Level Co-occurrence Matrix (GLCM). Each metric provides insight into the spatial distribution and relationship of pixel intensities within the image:

- **Contrast:**
  Measures the local variations in intensity. High contrast may indicate heterogeneous tissue.

- **Correlation:**
  Assesses the linear dependency between pixel intensities. Higher correlation can suggest more uniform textures.

- **Energy:**
  Also known as Angular Second Moment, it reflects textural uniformity. Higher energy implies smoother textures.

- **Homogeneity:**
  Indicates the closeness of the distribution of elements in the GLCM to the diagonal, reflecting texture smoothness.

- **Basic Statistics (Mean, Standard Deviation, Median):**
  These provide additional details about the intensity distribution, which can be useful for further clinical insights.

---

Proceed with the code cell below to run the texture analysis.


In [ ]:
!pip install numpy pandas matplotlib SimpleITK scikit-image jupyter

## Import Packages

In [6]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import SimpleITK as sitk
from skimage.feature.texture import graycomatrix, graycoprops
from skimage.util import img_as_ubyte

# Enable inline plotting in Jupyter
%matplotlib inline

## CT Processing Functions

In [7]:
def ensure_image(input_path, output_path):
    """
    Reads a image (PNG) using SimpleITK and ensures it has a supported pixel type.
    If not (e.g. it is 64-bit or int32), the image is cast to 32-bit float.
    The resulting image is saved to output_path.
    """
    try:
        image = sitk.ReadImage(input_path)
    except Exception as e:
        print(f"Error reading image {input_path} with SimpleITK: {e}")
        return None

    # Allowed pixel types for PNG writing.
    allowed_types = [sitk.sitkUInt8, sitk.sitkInt8, sitk.sitkUInt16, sitk.sitkInt16, sitk.sitkFloat32]
    if image.GetPixelID() not in allowed_types:
        image = sitk.Cast(image, sitk.sitkFloat32)
        print(f"Casting image {input_path} to 32-bit float (sitkFloat32).")
    try:
        sitk.WriteImage(image, output_path)
        print(f"Saved converted image to {output_path}")
    except Exception as e:
        print(f"Error writing image to {output_path}: {e}")
        return None
    return output_path

def analyze_texture(image_path):
    """
    Analyzes the texture of a CT or MRI image.
    The image is read with SimpleITK, converted to 8-bit (using img_as_ubyte),
    and then used to compute the Gray-Level Co-occurrence Matrix (GLCM) and
    derived texture features.
    Black frames (i.e. images with a maximum pixel value of 0) are skipped.
    """
    try:
        image = sitk.ReadImage(image_path)
        image_array = sitk.GetArrayFromImage(image)
        # Skip completely black images.
        if np.max(image_array) == 0:
            print(f"Skipping image {image_path} because it is a black frame.")
            return None

        image_array = img_as_ubyte(image_array)
        distances = [5]
        angles = [0, np.pi / 4, np.pi / 2, 3 * np.pi / 4]
        glcm = graycomatrix(image_array, distances=distances, angles=angles,
                             symmetric=True, normed=True)
        contrast = graycoprops(glcm, 'contrast')[0, 0]
        correlation = graycoprops(glcm, 'correlation')[0, 0]
        energy = graycoprops(glcm, 'energy')[0, 0]
        homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
        mean_val = np.mean(image_array)
        std_val = np.std(image_array)
        median_val = np.median(image_array)
        return {
            'contrast': contrast,
            'correlation': correlation,
            'energy': energy,
            'homogeneity': homogeneity,
            'mean': mean_val,
            'std': std_val,
            'median': median_val
        }
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return None

## Image Stack Analysis Function

In [9]:
def analyze_image_stack(image_directory, output_csv, converted_dir=None):
    """
    Analyzes a stack of PNG images in a directory and saves the results to a CSV file.
    Each file is converted (if needed) using the CT processing pipeline and then analyzed.
    Black frame images are skipped.
    """
    all_features = {}
    # Filter for PNG files.
    image_files = [f for f in os.listdir(image_directory) if f.lower().endswith('.png')]

    # Sort files based on a numerical part of the filename.
    def extract_number(filename):
        match = re.search(r'_(\d{4})\.png$', filename)
        return int(match.group(1)) if match else 0

    image_files.sort(key=extract_number)

    # If no converted directory is provided, make one inside image_directory.
    if converted_dir is None:
        converted_dir = os.path.join(image_directory, "CT_converted")
    if not os.path.exists(converted_dir):
        os.makedirs(converted_dir)
    for image_file in image_files:
        original_path = os.path.join(image_directory, image_file)
        converted_path = os.path.join(converted_dir, image_file)
        ct_path = ensure_image(original_path, converted_path)
        if ct_path is None:
            print(f"Skipping {original_path} due to conversion error.")
            continue
        features = analyze_texture(ct_path)
        if features is not None:
            all_features[image_file] = features

    # Save the results to CSV.
    df = pd.DataFrame.from_dict(all_features, orient='index')
    df.to_csv(output_csv, index=True)
    print(f"Texture analysis results saved to {output_csv}")
    return all_features

def show_images_with_analysis(converted_dir, all_features):
    """
    Displays CT images (from the converted folder) along with their texture analysis.
    """
    for image_file, features in all_features.items():
        image_path = os.path.join(converted_dir, image_file)
        try:
            image = sitk.ReadImage(image_path)
            image_array = sitk.GetArrayFromImage(image)
        except Exception as e:
            print(f"Error reading CT image {image_path} for display: {e}")
            continue
        plt.figure()
        plt.imshow(image_array, cmap='gray')
        plt.title(f"{image_file}\nContrast: {features['contrast']:.2f}, "
                  f"Correlation: {features['correlation']:.2f}, Energy: {features['energy']:.2f}, "
                  f"Homogeneity: {features['homogeneity']:.2f}")
        plt.axis('off')
        plt.show()

## Main Cell

In [11]:
if __name__ == "__main__":
    # Set the directory of the original PNG images.
    image_directory = "/path/to/your/png/files"  # Change this to your image directory!

    # Create a directory for converted images if it doesn't exist.
    converted_directory = os.path.join(image_directory, "/folder/_converted")
    if not os.path.exists(converted_directory):
        os.makedirs(converted_directory)

    # Define the CSV output file location.
    output_csv = os.path.join(converted_directory, "texture_analysis_results.csv")

    # Analyze the image stack using the CT pipeline.
    all_features = analyze_image_stack(image_directory, output_csv, converted_directory)
    show_images_with_analysis(converted_directory, all_features)

Error processing MRI image /Users/colehanan/Desktop/Group_2/MRI_2/MRI_2_0000.png: not a TIFF file b'\x89PNG'
Error processing MRI image /Users/colehanan/Desktop/Group_2/MRI_2/MRI_2_0001.png: not a TIFF file b'\x89PNG'
Error processing MRI image /Users/colehanan/Desktop/Group_2/MRI_2/MRI_2_0002.png: not a TIFF file b'\x89PNG'
Error processing MRI image /Users/colehanan/Desktop/Group_2/MRI_2/MRI_2_0003.png: not a TIFF file b'\x89PNG'
Error processing MRI image /Users/colehanan/Desktop/Group_2/MRI_2/MRI_2_0004.png: not a TIFF file b'\x89PNG'
Error processing MRI image /Users/colehanan/Desktop/Group_2/MRI_2/MRI_2_0005.png: not a TIFF file b'\x89PNG'
Error processing MRI image /Users/colehanan/Desktop/Group_2/MRI_2/MRI_2_0006.png: not a TIFF file b'\x89PNG'
Error processing MRI image /Users/colehanan/Desktop/Group_2/MRI_2/MRI_2_0007.png: not a TIFF file b'\x89PNG'
Error processing MRI image /Users/colehanan/Desktop/Group_2/MRI_2/MRI_2_0008.png: not a TIFF file b'\x89PNG'
Error processing MR